**Deep Learning Models**

In [35]:
from google.colab import drive
drive.mount('/content/drive') #Here we are going to extract our dataset into colab from from google drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

# Path to your CSV file
csv_file_path = '/content/drive/MyDrive/HateSpeechDataset.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)
df.head()

,Content,Label,Content_int
0,denial of normal the con be asked to comment o...,1,"[146715, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,..."
1,just by being able to tweet this insufferable ...,1,"[146715, 14, 15, 16, 17, 7, 18, 19, 20, 21, 22..."
2,that is retarded you too cute to be single tha...,1,"[146715, 28, 29, 30, 26, 31, 32, 7, 5, 33, 28,..."
3,thought of a real badass mongol style declarat...,1,"[146715, 35, 1, 24, 36, 37, 38, 39, 40, 1, 41,..."
4,afro american basho,1,"[146715, 46, 47, 48, 146714]"


**Importing Libraries**

In [12]:
import pandas as pd
import numpy as np
import re
import multiprocessing
#Here we are importing various libraries to use in further

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
%matplotlib inline

import nltk
from nltk.corpus import stopwords


from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

import string
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [13]:
# Path to your CSV file
csv_file_path = '/content/drive/MyDrive/HateSpeechDataset.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)
df.head()

,Content,Label,Content_int
0,denial of normal the con be asked to comment o...,1,"[146715, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,..."
1,just by being able to tweet this insufferable ...,1,"[146715, 14, 15, 16, 17, 7, 18, 19, 20, 21, 22..."
2,that is retarded you too cute to be single tha...,1,"[146715, 28, 29, 30, 26, 31, 32, 7, 5, 33, 28,..."
3,thought of a real badass mongol style declarat...,1,"[146715, 35, 1, 24, 36, 37, 38, 39, 40, 1, 41,..."
4,afro american basho,1,"[146715, 46, 47, 48, 146714]"


In [15]:
df.shape #Totaly we have 440906 rows of data

(440906, 3)

In [16]:
train = df.drop(['Content_int'], axis = 1)

In [17]:
train.head()


,Content,Label
0,denial of normal the con be asked to comment o...,1
1,just by being able to tweet this insufferable ...,1
2,that is retarded you too cute to be single tha...,1
3,thought of a real badass mongol style declarat...,1
4,afro american basho,1


**Data Preprocessing**

In [21]:

import nltk
import string
import re

# Download the 'stopwords' resource
nltk.download('stopwords')

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)



#Removing the stopwords from text
def split_into_words(text):
    # split into words by white space
    words = text.split()
    return words

def to_lower_case(words):
    # convert to lower case
    words = [word.lower() for word in words]
    return words

def remove_punctuation(words):
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    stripped = [re_punc.sub('', w) for w in words]
    return stripped

def keep_alphabetic(words):
    # remove remaining tokens that are not alphabetic
    words = [word for word in words if word.isalpha()]
    return words

def remove_stopwords(words):
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words

def to_sentence(words):
    # join words to a sentence
    return ' '.join(words)
def tweet(words):
    tweet_tokenizer = nltk.tokenize.TweetTokenizer(strip_handles=True,reduce_len=True)
    tweet = tweet_tokenizer.tokenize(words)
    return tweet

#Removing the noisy text
def denoise_text(text):
    words = split_into_words(text)
    words = to_lower_case(words)
    words = remove_punctuation(words)
    words = keep_alphabetic(words)
    words = remove_stopwords(words)
    return to_sentence(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [22]:
train['Content']=train['Content'].apply(denoise_text)

In [23]:
train

,Content,Label
0,denial normal con asked comment tragedies emot...,1
1,able tweet insufferable bullshit proves trump ...,1
2,retarded cute single life,1
3,thought real badass mongol style declaration w...,1
4,afro american basho,1
...,...,...
440901,crash another movie left field ask star wars e...,0
440902,debate first starting edit war,0
440903,removal reverted removal section believe misgu...,0
440904,unblocked eddie discussed user talkable accusi...,0


from matplotlib import pyplot as plt
import seaborn as sns
train.groupby('Label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [24]:
X = train.Content
Y = train.Label
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

**Dataset splitting into Training and Testing**

In [25]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [26]:
from keras import utils
max_words = 1000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = utils.pad_sequences(sequences,maxlen=max_len)

Building **RNN Model**

In [27]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

**RNN Model ** Structure

In [28]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

**Model Validation with accuarcy**

In [29]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
2343/2343 [==============================] - 1031s 438ms/step - loss: 0.3674 - accuracy: 0.8370 - val_loss: 0.3617 - val_accuracy: 0.8413
Epoch 2/10
2343/2343 [==============================] - 1096s 468ms/step - loss: 0.3461 - accuracy: 0.8462 - val_loss: 0.3426 - val_accuracy: 0.8478
Epoch 3/10
2343/2343 [==============================] - 1100s 469ms/step - loss: 0.3390 - accuracy: 0.8499 - val_loss: 0.3406 - val_accuracy: 0.8486
Epoch 4/10
2343/2343 [==============================] - 1019s 435ms/step - loss: 0.3345 - accuracy: 0.8516 - val_loss: 0.3386 - val_accuracy: 0.8498
Epoch 5/10
2343/2343 [==============================] - 1090s 465ms/step - loss: 0.3306 - accuracy: 0.8535 - val_loss: 0.3395 - val_accuracy: 0.8516


In [30]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = utils.pad_sequences(test_sequences,maxlen=max_len)

In [31]:
accr = model.evaluate(test_sequences_matrix,Y_test)

2067/2067 [==============================] - 105s 51ms/step - loss: 0.3323 - accuracy: 0.8539


In [32]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.332
  Accuracy: 0.854
